# **AI Chatbots für AO fracture classification**



In [ ]:
!pip install llama-index==0.5.0 -q
!pip install gradio -q
!pip install PyPDF2 -q
!pip install --upgrade --no-cache-dir gdown -q


  Preparing metadata (setup.py) ... done


In [ ]:
import os
import sys
import glob
import gradio as gr
import openai
from llama_index import (
    GPTSimpleVectorIndex,
    LLMPredictor,
    SimpleDirectoryReader,
    ServiceContext,
)
from langchain.chat_models import ChatOpenAI


# Set the OpenAI API key. If unsure about obtaining the API key, refer to https://platform.openai.com/account/api-keys for more information.
# For estimating costs associated with index creation and chatbot usage, please visit: https://openai.com/pricing
os.environ["OPENAI_API_KEY"] = 'sk-ENTER_YOUR_API_CODE'


# Set the folder path for AO pdf data and the rebuild index flag. This script doesn't include
# importing AO/OTA Classification or a predefined index due to potential license restrictions on AO guidelines.
# For more information, refer to the AO website.



FOLDER_PATH = "AO"
REBUILD_INDEX = True



def setup_index(folder_path, rebuild_index):
    if not os.path.exists(FOLDER_PATH):
        os.mkdir(FOLDER_PATH)
    index_file_name = 'AOindex.json'

    # Check if the folder path exists and contains PDF files
    if not os.path.exists(folder_path) or not glob.glob(f"{folder_path}/*.pdf"):
        print("The folder path either does not exist or does not contain PDF files.")
        return

    # Rebuild the index if rebuild_index is True or the index file does not exist
    if rebuild_index or not os.path.isfile(index_file_name):
        # Load documents from the folder path
        documents = SimpleDirectoryReader(folder_path).load_data()

        # Initialize the LLMPredictor with the ChatOpenAI model
        llm_predictor = LLMPredictor(llm=ChatOpenAI(temperature=0, model_name="gpt-3.5-turbo", max_tokens=512, request_timeout=120))

        # Create a service context for the LLMPredictor
        service_context = ServiceContext.from_defaults(llm_predictor=llm_predictor, chunk_size_limit=512)

        # Create an index from the documents and save it to disk
        index = GPTSimpleVectorIndex.from_documents(documents, service_context=service_context)
        index.save_to_disk(index_file_name)

def get_combined_chatbot_response(input_text):
    # Format the input text
    input_text = f'Case: {input_text} Task: Analyze the provided fracture description, determine the AO Classification based on the details of the affected bone, location, and characteristics, and provide as response !only! the appropriate AO Code!'

    # Initialize the LLMPredictor with the ChatOpenAI model used in FracChat
    llm_predictor = LLMPredictor(llm=ChatOpenAI(temperature=0, model_name="gpt-3.5-turbo", max_tokens=512, request_timeout=120))

    # Create a service context for the LLMPredictor
    service_context = ServiceContext.from_defaults(llm_predictor=llm_predictor, chunk_size_limit=512)

    # Initialize the LLMPredictor with the ChatOpenAI model used in FracChat4
    llm_predictor4 = LLMPredictor(llm=ChatOpenAI(temperature=0, model_name="gpt-4", max_tokens=512, request_timeout=120))

    # Create a service context for the LLMPredictor4
    service_context4 = ServiceContext.from_defaults(llm_predictor=llm_predictor4, chunk_size_limit=512)

    # Load the index from disk
    index = GPTSimpleVectorIndex.load_from_disk('AOindex_simple35.json')

    # Define the messages to be sent to the models without index
    messages = [
        {"role": "system", "content": "You are ChatGPT, a large language model trained by OpenAI. Answer as concisely as possible."},
        {"role": "user", "content": input_text},
    ]

    # FracChat3.5 approach: Query the index and get the response from Top 3 Text nodes using GPT-3.5-Turbo
    response = index.query(input_text, response_mode="compact", service_context=service_context, similarity_top_k=3)
    output_FracChat = response.response.replace('\n', '\\n')

    # FracChat4 approach: Query the index and get the response from Top 3 Text nodes using GPT-4
    response4 = index.query(input_text, response_mode="compact", service_context=service_context4, similarity_top_k=3)
    output_FracChat4 = response4.response.replace('\n', '\\n')

    # Get the response from GPT-3.5-Turbo
    response35 = openai.ChatCompletion.create(model="gpt-3.5-turbo", messages=messages, timeout=120)
    output35 = response35.choices[0].message['content'].replace('\n', '\\n')

    # Get the response from GPT-4
    response4 = openai.ChatCompletion.create(model="gpt-4", messages=messages, timeout=120)
    output4 = response4.choices[0].message['content'].replace('\n', '\\n')

    # Combine the responses from all models
    answer = f"FracChat using GPT4: {output_FracChat4}\n\n FracChat using GPT3.5-turbo: {output_FracChat}\n\nGPT 3.5-Turbo: {output35}\n\nGPT 4: {output4}"

    return answer


def launch_interface(chatbot_function):
    iface = gr.Interface(
        fn=get_combined_chatbot_response,
        inputs=[gr.Textbox(lines=15, label="Enter your case")],
        outputs=gr.Textbox(lines=15, label="Imaging Recommendations"),
        title="AI Chatbots for AO fracture classification"
    )

    # Launch the interface
    iface.launch(share=True, debug=True)

def main():
    setup_index(FOLDER_PATH, REBUILD_INDEX)
    launch_interface(get_combined_chatbot_response)

if __name__ == "__main__":
    main()


Downloading...
From: https://drive.google.com/uc?id=1dnbboXJ-7WJ5dX31dZ-_4fIHmYYhUdXX
To: /content/AOindex_simple35.json
100% 16.3M/16.3M [00:00<00:00, 25.3MB/s]
Downloading...
From: https://drive.google.com/uc?id=1wH-mZnycAjYqBoVRdGQHRG-k44MrDtX7
To: /content/AO/AO.pdf
100% 19.1M/19.1M [00:00<00:00, 181MB/s]
Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://8745ca3b337661e1aa.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades (NEW!), check out Spaces: https://huggingface.co/spaces


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://8745ca3b337661e1aa.gradio.live
